ASSIGNMENT-04
  

*   RAJPRIYA R - GCTC1918130
*   KIRUBA R - GCTC1918119
*   RESHMA M - GCTC1918131
*   NARMADHA A - GCTC1918126




In [1]:
import pandas as pd
import numpy as np
import tensorflow

LOAD THE DATASET

In [4]:
data = pd.read_csv("/content/drive/MyDrive/ASSIGNMENT_04/spam.csv", encoding="ISO-8859-1")

In [5]:
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [6]:
data.describe().T

,count,unique,top,freq
v1,5572,2,ham,4825
v2,5572,5169,"Sorry, I'll call later",30
Unnamed: 2,50,43,"bt not his girlfrnd... G o o d n i g h t . . .@""",3
Unnamed: 3,12,10,"MK17 92H. 450Ppw 16""",2
Unnamed: 4,6,5,"GNT:-)""",2


In [7]:
data.shape

(5572, 5)

In [8]:
data.isnull().sum()

v1               0
v2               0
Unnamed: 2    5522
Unnamed: 3    5560
Unnamed: 4    5566
dtype: int64

In [9]:
data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)

In [48]:
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer


In [49]:
X = data.v2
Y = data.v1
le = LabelEncoder()
Y = le.fit_transform(Y)

In [50]:
Y = Y.reshape(-1,1)

CREATE THE MODEL

In [51]:
from sklearn.model_selection import train_test_split
from keras.layers import Dense , LSTM , Embedding , Dropout , Activation , Flatten

In [52]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.2)

In [53]:
from keras.models import Sequential
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop

In [54]:
max_word = 1000
max_len = 250
token = Tokenizer(num_words = max_word)
token.fit_on_texts(X_train)
sequences = token.texts_to_sequences(X_train)
seq_matrix = sequence.pad_sequences(sequences , maxlen = max_len)

ADDING LAYERS


*   LSTM
*   DENSE
*   OUTPUT





In [55]:
model = Sequential()
model.add(Embedding(max_word , 32 , input_length = max_len))
model.add(LSTM(64))
model.add(Flatten())

model.add(Dense(250, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(120, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

COMPILE THE MODEL

In [56]:
model.compile(loss = 'binary_crossentropy' , optimizer = 'RMSprop' , metrics = 'accuracy')
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 250, 32)           32000     
                                                                 
 lstm_1 (LSTM)               (None, 64)                24832     
                                                                 
 flatten_1 (Flatten)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 250)               16250     
                                                                 
 dropout_1 (Dropout)         (None, 250)               0         
                                                                 
 dense_4 (Dense)             (None, 120)               30120     
                                                                 
 dense_5 (Dense)             (None, 1)                

FIT THE MODEL

In [57]:
model.fit(seq_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/10
28/28 [==============================] - 14s 413ms/step - loss: 0.3193 - accuracy: 0.8749 - val_loss: 0.1319 - val_accuracy: 0.9742
Epoch 2/10
28/28 [==============================] - 11s 391ms/step - loss: 0.0832 - accuracy: 0.9795 - val_loss: 0.0818 - val_accuracy: 0.9731


SAVE THE MODEL

In [58]:
model.save("spam.h5")

TEST THE MODEL

In [59]:
from tensorflow.keras.models import load_model

In [60]:
model1 = load_model("spam.h5")

In [61]:
test_seq = token.texts_to_sequences(X_test)
test_seq_matrix = sequence.pad_sequences(test_seq,maxlen=max_len)

In [62]:
scores = model.evaluate(test_seq_matrix, Y_test, verbose=0)
scores

[0.07071194797754288, 0.9829596281051636]